In [106]:
import pandas as pd
from dotenv import load_dotenv

from pydantic import BaseModel
from langchain_google_genai import ChatGoogleGenerativeAI

from api_helpers.clients import get_postgres_client


load_dotenv(
    dotenv_path="/Users/tomwattley/App/racing-api-project/racing-api-project/libraries/api-helpers/src/api_helpers/.env"

)

from api_helpers.config import config

pg = get_postgres_client()


In [107]:
from dataclasses import dataclass
from datetime import datetime, timedelta
from time import sleep
from typing import Literal

import betfairlightweight
import numpy as np
import pandas as pd
import requests
from api_helpers.helpers.logging_config import D, I
from api_helpers.helpers.time_utils import get_uk_time_now, make_uk_time_aware

MARKET_FILTER = betfairlightweight.filters.market_filter(
    event_type_ids=["7"],
    market_countries=["GB"],
    market_type_codes=["WIN", "PLACE"],
    market_start_time={
        "from": ((datetime.now()) - timedelta(hours=1)).strftime("%Y-%m-%dT%TZ"),
        "to": (datetime.now())
        .replace(hour=23, minute=59, second=0, microsecond=0)
        .strftime("%Y-%m-%dT%TZ"),
    },
)


MARKET_PROJECTION = [
    "COMPETITION",
    "EVENT",
    "EVENT_TYPE",
    "MARKET_START_TIME",
    "MARKET_DESCRIPTION",
    "RUNNER_DESCRIPTION",
    "RUNNER_METADATA",
]

PRICE_PROJECTION = betfairlightweight.filters.price_projection(
    price_data=betfairlightweight.filters.price_data(ex_all_offers=True)
)


@dataclass(frozen=True)
class BetFairCancelOrders:
    market_ids: list[str]


@dataclass(frozen=True)
class BetFairOrder:
    size: float
    price: float
    selection_id: str
    market_id: str
    side: Literal["BACK", "LAY"]
    strategy: str


@dataclass
class BetfairCredentials:
    username: str
    password: str
    app_key: str
    certs_path: str


@dataclass
class BetfairHistoricalDataParams:
    from_day: int
    from_month: int
    from_year: int
    to_day: int
    to_month: int
    to_year: int
    market_types_collection: list[str]
    countries_collection: list[str]
    file_type_collection: list[str]


@dataclass
class OrderResult:
    success: bool
    message: str
    size_matched: float | None = None
    average_price_matched: float | None = None

    def __bool__(self) -> bool:
        """Allow the result to be used in boolean contexts"""
        return self.success


class BetFairCashOut:
    def cash_out(self, data: pd.DataFrame) -> list[BetFairOrder | None]:
        cash_out_orders = []
        for selection in data["selection_id"].unique():
            selection_df = data[data["selection_id"] == selection]
            if list(selection_df["selection_type"].unique()) == ["BACK", "LAY"]:
                cash_out_orders.extend(
                    self._handle_back_and_lay_matched_bets(selection_df)
                )
            elif list(selection_df["selection_type"].unique()) == ["BACK"]:
                cash_out_orders.extend(
                    self._handle_single_matched_back_bets(selection_df)
                )
            elif list(selection_df["selection_type"].unique()) == ["LAY"]:
                cash_out_orders.extend(
                    self._handle_single_matched_lay_bets(selection_df)
                )
            else:
                raise ValueError("Unidentified bet type")
        return cash_out_orders

    @staticmethod
    def _create_average_lay_odds(data: pd.DataFrame) -> pd.DataFrame:
        data = data.assign(
            ave_lay_odds=(
                (
                    (data["lay_price_1"] * data["lay_price_1_depth"])
                    + (data["lay_price_2"] * data["lay_price_2_depth"])
                )
                / (data["lay_price_1_depth"] + data["lay_price_2_depth"])
            ).round(2),
        )
        return data

    @staticmethod
    def _create_average_back_odds(data: pd.DataFrame) -> pd.DataFrame:
        data = data.assign(
            ave_back_odds=(
                (
                    (data["back_price_1"] * data["back_price_1_depth"])
                    + (data["back_price_2"] * data["back_price_2_depth"])
                )
                / (data["back_price_1_depth"] + data["back_price_2_depth"])
            ).round(2),
        )
        return data

    @staticmethod
    def _create_cash_out_odds(data: pd.DataFrame) -> pd.DataFrame:
        if "ave_back_odds" in data.columns and "ave_lay_odds" in data.columns:
            return data.assign(
                cash_out_odds=np.where(
                    data["selection_type"] == "BACK",
                    data["ave_back_odds"],
                    data["ave_lay_odds"],
                )
            )
        elif "ave_back_odds" not in data.columns and "ave_lay_odds" in data.columns:
            return data.assign(cash_out_odds=data["ave_lay_odds"])
        elif "ave_back_odds" in data.columns and "ave_lay_odds" not in data.columns:
            return data.assign(cash_out_odds=data["ave_back_odds"])
        else:
            raise ValueError("No average odds found")

    @staticmethod
    def _merge_back_and_lay_data(data: pd.DataFrame) -> pd.DataFrame:
        return pd.merge(
            data[data["selection_type"] == "BACK"][
                [
                    "market_id",
                    "selection_id",
                    "back_price_2",
                    "average_price_matched",
                    "size_matched",
                    "cash_out_odds",
                ]
            ],
            data[data["selection_type"] == "LAY"][
                [
                    "market_id",
                    "selection_id",
                    "lay_price_2",
                    "average_price_matched",
                    "size_matched",
                    "cash_out_odds",
                ]
            ],
            on=[
                "market_id",
                "selection_id",
            ],
            how="left",
            suffixes=["_back", "_lay"],
        )

    @staticmethod
    def _create_bet_side(data: pd.DataFrame) -> pd.DataFrame:
        return data.assign(
            lay_liability=(
                (data["average_price_matched_lay"] - 1) * data["size_matched_lay"]
            )
            - data["size_matched_back"],
            back_winnings=(
                (data["average_price_matched_back"] - 1) * data["size_matched_back"]
            )
            - data["size_matched_lay"],
            risk_diff=lambda x: x["back_winnings"] - x["lay_liability"],
            cash_out_selection_type=lambda x: np.where(
                x["risk_diff"] > 0, "LAY", "BACK"
            ),
        ).drop(columns=["risk_diff", "lay_liability", "back_winnings"])

    @staticmethod
    def _alternate_bet_side(data: pd.DataFrame) -> pd.DataFrame:
        return data.assign(
            cash_out_selection_type=np.where(
                data["selection_type"] == "LAY", "BACK", "LAY"
            ),
        )

    @staticmethod
    def _get_cash_out_stake_back_and_lay(data: pd.DataFrame) -> pd.DataFrame:
        return data.assign(
            cash_out_stake=np.select(
                [
                    data["cash_out_selection_type"] == "LAY",
                    data["cash_out_selection_type"] == "BACK",
                ],
                [
                    (
                        (
                            (
                                data["size_matched_back"]
                                * (data["average_price_matched_back"] - 1)
                                - data["size_matched_lay"]
                                * (data["average_price_matched_lay"] - 1)
                            )
                            + (data["size_matched_back"] - data["size_matched_lay"])
                        )
                        / data["cash_out_odds_lay"]
                    ).round(2),
                    (
                        (
                            (
                                data["size_matched_lay"]
                                * (data["average_price_matched_lay"] - 1)
                                - data["size_matched_back"]
                                * (data["average_price_matched_back"] - 1)
                            )
                            + (data["size_matched_lay"] - data["size_matched_back"])
                        )
                        / data["cash_out_odds_back"]
                    ).round(2),
                ],
                default=np.nan,
            ),
        )

    @staticmethod
    def _get_cash_out_stake(data: pd.DataFrame) -> pd.DataFrame:
        return data.assign(
            cash_out_stake=(
                (data["size_matched"] * data["average_price_matched"])
                / data["cash_out_odds"]
            ).round(2),
        )

    @staticmethod
    def _get_cash_out_odds(data: pd.DataFrame) -> pd.DataFrame:
        return data.assign(
            cash_out_odds=np.select(
                [
                    data["cash_out_selection_type"] == "LAY",
                    data["cash_out_selection_type"] == "BACK",
                ],
                [data["lay_price_2"], data["back_price_2"]],
                default=np.nan,
            ),
        )

    @staticmethod
    def _create_bet_orders(data: pd.DataFrame) -> BetFairOrder:
        data = data[data["cash_out_stake"] > 2]
        return [
            BetFairOrder(
                size=float(data_dict["cash_out_stake"]),
                price=float(data_dict["cash_out_odds"]),
                selection_id=str(data_dict["selection_id"]),
                market_id=str(data_dict["market_id"]),
                side=str(data_dict["cash_out_selection_type"]),
                strategy="cash_out",
            )
            for data_dict in data.to_dict("records")
        ]

    @staticmethod
    def _handle_back_and_lay_matched_bets(data: pd.DataFrame) -> BetFairOrder:
        return (
            data.pipe(BetFairCashOut._create_average_back_odds)
            .pipe(BetFairCashOut._create_average_lay_odds)
            .pipe(BetFairCashOut._create_cash_out_odds)
            .pipe(BetFairCashOut._merge_back_and_lay_data)
            .pipe(BetFairCashOut._create_bet_side)
            .pipe(BetFairCashOut._get_cash_out_stake_back_and_lay)
            .pipe(BetFairCashOut._get_cash_out_odds)
            .pipe(BetFairCashOut._create_bet_orders)
        )

    @staticmethod
    def _handle_single_matched_back_bets(data: pd.DataFrame) -> BetFairOrder:
        return (
            data.pipe(BetFairCashOut._create_average_lay_odds)
            .pipe(BetFairCashOut._create_cash_out_odds)
            .pipe(BetFairCashOut._alternate_bet_side)
            .pipe(BetFairCashOut._get_cash_out_stake)
            .pipe(BetFairCashOut._get_cash_out_odds)
            .pipe(BetFairCashOut._create_bet_orders)
        )

    @staticmethod
    def _handle_single_matched_lay_bets(data: pd.DataFrame) -> BetFairOrder:
        return (
            data.pipe(BetFairCashOut._create_average_back_odds)
            .pipe(BetFairCashOut._create_cash_out_odds)
            .pipe(BetFairCashOut._alternate_bet_side)
            .pipe(BetFairCashOut._get_cash_out_stake)
            .pipe(BetFairCashOut._get_cash_out_odds)
            .pipe(BetFairCashOut._create_bet_orders)
        )


class BetFairClient:
    """
    Betfair client
    """

    def __init__(
        self, credentials: BetfairCredentials, betfair_cash_out: BetFairCashOut
    ):
        self.credentials = credentials
        self.betfair_cash_out = betfair_cash_out
        self.trading_client: betfairlightweight.APIClient | None = None

    def login(self):
        if self.trading_client is None or self.trading_client.session_expired:
            I("Logging into Betfair...")
            self.trading_client = betfairlightweight.APIClient(
                username=self.credentials.username,
                password=self.credentials.password,
                app_key=self.credentials.app_key,
                certs=self.credentials.certs_path,
            )
            self.trading_client.login(session=requests)
            I("Logged into Betfair!")

    def check_session(self):
        if self.trading_client is None or self.trading_client.session_expired:
            I("Betfair session expired")
            self.login()

    def logout(self):
        if self.trading_client is not None:
            self.trading_client.logout()
            I("Logged out of Betfair")

    def create_market_data(self) -> pd.DataFrame:
        self.check_session()
        markets, runners = self._create_markets_and_runners()
        return self._process_combined_market_data(markets, runners)

    def _get_single_race_markets(self, market_ids: list[str]):
        self.check_session()
        markets = self.trading_client.betting.list_market_catalogue(
            filter=betfairlightweight.filters.market_filter(
                market_ids=market_ids,
            ),
            market_projection=MARKET_PROJECTION,
            max_results=1000,
        )

        D(f"Found {len(markets)} markets")
        runners = {
            runner.selection_id: runner.runner_name
            for market in markets
            for runner in market.runners
        }

        return markets, runners

    def create_single_market_data(self, market_ids: list[str]) -> pd.DataFrame:
        self.check_session()
        markets, runners = self._get_single_race_markets(market_ids)
        data = self._process_combined_market_data(markets, runners)
        return data.rename(
            columns={
                "horse_win": "horse_name",
                "todays_betfair_selection_id": "selection_id",
                "last_traded_price_win": "betfair_win_sp",
                "last_traded_price_place": "betfair_place_sp",
            }
        )

    def create_merged_single_market_data(self, market_ids: list[str]) -> pd.DataFrame:
        self.check_session()
        markets, runners = self._get_single_race_markets(market_ids)
        data = self._process_combined_market_data(markets, runners)
        data = data.rename(
            columns={
                "horse_win": "horse_name",
                "last_traded_price_win": "betfair_win_sp",
                "last_traded_price_place": "betfair_place_sp",
            }
        )
        return pd.merge(
            data[data["market"] == "WIN"],
            data[data["market"] == "PLACE"],
            on=["race_time", "course", "todays_betfair_selection_id"],
            suffixes=("_win", "_place"),
        ).rename(
            columns={
                "horse_win": "horse_name",
                "todays_betfair_selection_id": "horse_id",
                "last_traded_price_win": "betfair_win_sp",
                "last_traded_price_place": "betfair_place_sp",
            }
        )

    def create_market_order_data(self, market_ids: list[str]) -> pd.DataFrame:
        self.check_session()
        markets, runners = self._get_single_race_markets(market_ids)
        data = self._process_combined_market_data(markets, runners)
        data = data.rename(
            columns={
                "todays_betfair_selection_id": "selection_id",
                "market": "market_type",
                "status": "runner_status",
            }
        )
        return data[data["runner_status"] == "ACTIVE"]

    def _create_markets_and_runners(self):
        self.check_session()
        markets = self.trading_client.betting.list_market_catalogue(
            filter=MARKET_FILTER,
            market_projection=MARKET_PROJECTION,
            max_results=1000,
        )
        I(f"Found {len(markets)} markets")
        runners = {
            runner.selection_id: runner.runner_name
            for market in markets
            for runner in market.runners
        }

        return markets, runners

    def get_min_and_max_race_times(self) -> tuple[pd.Timestamp, pd.Timestamp]:
        self.check_session()
        markets, _ = self._create_markets_and_runners()
        start_times = [market.market_start_time for market in markets]
        if not start_times:
            raise ValueError("No markets found")
        return make_uk_time_aware(min(start_times)), make_uk_time_aware(
            max(start_times)
        )

    def _process_combined_market_data(self, markets, runners) -> pd.DataFrame:
        self.check_session()
        combined_data = []

        for market in markets:
            uk_now = get_uk_time_now()
            if make_uk_time_aware(market.market_start_time) <= uk_now:
                I(f"Skipping market {market.market_id} already started")
                continue

            market_book = self.trading_client.betting.list_market_book(
                market_ids=[market.market_id],
                price_projection=PRICE_PROJECTION,
            )
            market_type = market.description.market_type

            for book in market_book:
                for runner in book.runners:
                    runner_data = {
                        "race_time": make_uk_time_aware(market.market_start_time),
                        "market": market_type,
                        "race": market.market_name,
                        "course": market.event.venue,
                        "horse": runners[runner.selection_id],
                        "status": runner.status,
                        "market_id": market.market_id,
                        "todays_betfair_selection_id": runner.selection_id,
                        "last_traded_price": runner.last_price_traded,
                        "total_matched": runner.total_matched,
                    }

                    if runner.status == "ACTIVE":
                        for i, price in enumerate(runner.ex.available_to_back[:5]):
                            runner_data[f"back_price_{i + 1}"] = price.price
                            runner_data[f"back_price_{i + 1}_depth"] = int(
                                round(price.size, 0)
                            )

                        for i, price in enumerate(runner.ex.available_to_lay[:5]):
                            runner_data[f"lay_price_{i + 1}"] = price.price
                            runner_data[f"lay_price_{i + 1}_depth"] = int(
                                round(price.size, 0)
                            )

                    combined_data.append(runner_data)

        data = pd.DataFrame(combined_data)

        data["total_matched_event"] = (
            data.groupby("market_id")["total_matched"]
            .transform("sum")
            .round(0)
            .astype(int)
        )

        data["percent_back_win_chance"] = 100 / data["back_price_1"]
        data["percent_lay_win_chance"] = 100 / data["lay_price_1"]

        data["percent_back_win_book"] = (
            data.groupby("market_id")["percent_back_win_chance"]
            .transform("sum")
            .round(0)
            .astype(int)
        )
        data["percent_lay_win_book"] = (
            data.groupby("market_id")["percent_lay_win_chance"]
            .transform("sum")
            .round(0)
            .astype(int)
        )
        data["market_width"] = (
            data["percent_back_win_book"] - data["percent_lay_win_book"]
        )

        return data[
            [
                "race_time",
                "market",
                "race",
                "course",
                "horse",
                "status",
                "market_id",
                "todays_betfair_selection_id",
                "last_traded_price",
                "total_matched",
                "back_price_1",
                "back_price_1_depth",
                "back_price_2",
                "back_price_2_depth",
                "back_price_3",
                "back_price_3_depth",
                "back_price_4",
                "back_price_4_depth",
                "back_price_5",
                "back_price_5_depth",
                "lay_price_1",
                "lay_price_1_depth",
                "lay_price_2",
                "lay_price_2_depth",
                "lay_price_3",
                "lay_price_3_depth",
                "lay_price_4",
                "lay_price_4_depth",
                "lay_price_5",
                "lay_price_5_depth",
                "total_matched_event",
                "percent_back_win_book",
                "percent_lay_win_book",
                "market_width",
            ]
        ]

    def place_order(
        self,
        betfair_order: BetFairOrder,
        max_retries: int = 3,
        retry_delay: float = 1.0,
    ) -> OrderResult:
        """
        Place a betfair order with retry logic for network failures.

        Args:
            betfair_order: The order to place
            max_retries: Maximum number of retry attempts (default: 3)
            retry_delay: Delay between retries in seconds (default: 1.0)

        Returns:
            OrderResult: Contains success status, message, response data, bet_id, and matching info
        """

        for attempt in range(max_retries + 1):
            try:
                self.check_session()
                D(
                    f"Placing order (attempt {attempt + 1}/{max_retries + 1}) - {betfair_order}"
                )

                response = self.trading_client.betting.place_orders(
                    market_id=betfair_order.market_id,
                    customer_strategy_ref="trader",
                    instructions=[
                        {
                            "orderType": "LIMIT",
                            "selectionId": betfair_order.selection_id,
                            "side": betfair_order.side,
                            "limitOrder": {
                                "price": betfair_order.price,
                                "persistenceType": "LAPSE",
                                "size": betfair_order.size,
                            },
                        }
                    ],
                )

                response_dict = (
                    response.__dict__
                    if hasattr(response, "__dict__")
                    else str(response)
                )

                size_matched = response_dict["_data"]["instructionReports"][0][
                    "sizeMatched"
                ]
                average_price_matched = response_dict["_data"]["instructionReports"][0][
                    "averagePriceMatched"
                ]

                return OrderResult(
                    success=True,
                    message="Order placed successfully",
                    size_matched=size_matched,
                    average_price_matched=average_price_matched,
                )

            except (
                ConnectionError,
                TimeoutError,
                requests.exceptions.RequestException,
            ) as network_error:
                I(f"Network error on attempt {attempt + 1}: {network_error}")
                if attempt < max_retries:
                    I(f"Retrying in {retry_delay} seconds...")
                    sleep(retry_delay)
                    retry_delay *= 1.5  # Exponential backoff
                    continue
                else:
                    return OrderResult(
                        success=False,
                        message=f"Failed after {max_retries + 1} attempts due to network error: {network_error}",
                    )

            except Exception as e:
                error_msg = f"Unexpected error placing order: {e}"
                I(error_msg)
                return OrderResult(success=False, message=error_msg)

        # This should never be reached, but just in case
        return OrderResult(
            success=False, message="Order placement failed for unknown reason"
        )

    def place_orders(self, betfair_orders: list[BetFairOrder]) -> list[OrderResult]:
        self.check_session()
        orders = []
        for order in betfair_orders:
            result = self.place_order(order)
            orders.append(result)
        return orders

    def cancel_orders(self, betfair_cancel_orders: BetFairCancelOrders):
        self.check_session()
        for id in betfair_cancel_orders.market_ids:
            I(f"Cancelling orders for market {id}")
            try:
                self.trading_client.betting.cancel_orders(market_id=id)
            except Exception as e:
                I(f"Error cancelling orders for market {id}: {e}")

    def cancel_all_orders(self):
        self.check_session()
        self.trading_client.betting.cancel_orders()

    def get_current_orders(self, market_ids: list[str] = None):
        self.check_session()
        current_orders = pd.DataFrame(
            self.trading_client.betting.list_current_orders().__dict__["_data"][
                "currentOrders"
            ]
        )
        current_order_columns = [
            "bet_id",
            "market_id",
            "selection_id",
            "selection_type",
            "execution_status",
            "placed_date",
            "matched_date",
            "average_price_matched",
            "customer_strategy_ref",
            "size_matched",
            "size_remaining",
            "size_lapsed",
            "size_cancelled",
            "size_voided",
            "price_size",
        ]
        if current_orders.empty:
            return pd.DataFrame(columns=current_order_columns)
        else:
            if "customerStrategyRef" not in current_orders.columns:
                current_orders["customerStrategyRef"] = "UI"
            if market_ids:
                current_orders = current_orders[
                    current_orders["marketId"].isin(market_ids)
                ]
            return (
                current_orders.rename(
                    columns={
                        "betId": "bet_id",
                        "marketId": "market_id",
                        "selectionId": "selection_id",
                        "side": "selection_type",
                        "status": "execution_status",
                        "placedDate": "placed_date",
                        "matchedDate": "matched_date",
                        "averagePriceMatched": "average_price_matched",
                        "customerStrategyRef": "customer_strategy_ref",
                        "sizeMatched": "size_matched",
                        "sizeRemaining": "size_remaining",
                        "sizeLapsed": "size_lapsed",
                        "sizeCancelled": "size_cancelled",
                        "sizeVoided": "size_voided",
                        "priceSize": "price_size",
                    }
                )
                .filter(items=current_order_columns)
                .assign(
                    customer_strategy_ref=lambda x: x["customer_strategy_ref"].fillna(
                        "UI"
                    )
                )
            ).pipe(
                BetFairClient.expand_price_size,
            )

    def get_current_orders_with_market_data(self):
        current_orders = self.get_current_orders()
        market_data = self.create_market_order_data(
            list(current_orders["market_id"].unique())
        )
        return pd.merge(
            current_orders, market_data, on=["market_id", "selection_id"], how="left"
        )

    def get_matched_orders(self, market_ids: list[str] = None):
        self.cancel_orders(BetFairCancelOrders(market_ids=market_ids))
        current_orders = self.get_current_orders(market_ids)
        assert_current_orders = current_orders[
            current_orders["execution_status"] == "EXECUTION_COMPLETE"
        ]

        if len(assert_current_orders) != len(current_orders):
            raise ValueError("Some orders have not been cleared")

        grouping_cols = ["market_id", "selection_id", "selection_type"]

        current_orders = current_orders.assign(
            sum_matched=lambda x: x["average_price_matched"] * x["size_matched"],
            horse_sum_matched=lambda x: x.groupby(grouping_cols)[
                "sum_matched"
            ].transform("sum"),
            horse_staked_matched=lambda x: x.groupby(grouping_cols)[
                "size_matched"
            ].transform("sum"),
            average_horse_odds_matched=lambda x: x["horse_sum_matched"]
            / x["horse_staked_matched"],
        )
        current_orders = current_orders.assign(
            average_horse_odds_matched=current_orders[
                "average_horse_odds_matched"
            ].round(2),
        )
        current_orders = (
            current_orders.drop(
                columns=[
                    "average_price_matched",
                    "size_matched",
                    "sum_matched",
                ]
            )
            .rename(
                columns={
                    "horse_staked_matched": "size_matched",
                    "average_horse_odds_matched": "average_price_matched",
                }
            )
            .filter(
                items=[
                    "market_id",
                    "selection_id",
                    "selection_type",
                    "average_price_matched",
                    "size_matched",
                    "customer_strategy_ref",
                ]
            )
        )
        return current_orders.drop_duplicates(subset=grouping_cols)

    def _process_cleared_orders(self, cleared_orders):
        if not cleared_orders.orders:
            I("No cleared orders found")
            return pd.DataFrame()

        # Convert to DataFrame
        orders_data = []
        for order in cleared_orders.orders:
            orders_data.append(
                {
                    "bet_count": getattr(order, "bet_count", None),
                    "bet_id": getattr(order, "bet_id", None),
                    "bet_outcome": getattr(order, "bet_outcome", None),
                    "customer_order_ref": getattr(order, "customer_order_ref", None),
                    "customer_strategy_ref": getattr(
                        order, "customer_strategy_ref", None
                    ),
                    "event_id": getattr(order, "event_id", None),
                    "event_type_id": getattr(order, "event_type_id", None),
                    "handicap": getattr(order, "handicap", None),
                    "last_matched_date": getattr(order, "last_matched_date", None),
                    "market_id": getattr(order, "market_id", None),
                    "order_type": getattr(order, "order_type", None),
                    "persistence_type": getattr(order, "persistence_type", None),
                    "placed_date": getattr(order, "placed_date", None),
                    "price_matched": getattr(order, "price_matched", None),
                    "price_reduced": getattr(order, "price_reduced", None),
                    "price_requested": getattr(order, "price_requested", None),
                    "profit": getattr(order, "profit", None),
                    "commission": getattr(order, "commission", None),
                    "selection_id": getattr(order, "selection_id", None),
                    "settled_date": getattr(order, "settled_date", None),
                    "side": getattr(order, "side", None),
                    "size_settled": getattr(order, "size_settled", None),
                    "size_cancelled": getattr(order, "size_cancelled", None),
                    "item_description": getattr(order, "item_description", None),
                }
            )

        return pd.DataFrame(orders_data)

    def get_past_orders_by_date_range(
        self, from_date: str, to_date: str
    ) -> pd.DataFrame:
        self.check_session()
        cleared_orders = self.trading_client.betting.list_cleared_orders(
            settled_date_range={"from": from_date, "to": to_date},
        )
        return self._process_cleared_orders(cleared_orders)

    def get_past_orders_by_market_id(
        self, market_ids: list[str] = None
    ) -> pd.DataFrame:
        self.check_session()
        cleared_orders = self.trading_client.betting.list_cleared_orders(
            market_ids=market_ids,
        )
        return self._process_cleared_orders(cleared_orders)

    @staticmethod
    def _get_market_ids_for_remaining_cash_out_bets(data: pd.DataFrame) -> list[str]:
        back_subset = data[data["selection_type"] == "BACK"]
        lay_subset = data[data["selection_type"] == "LAY"]

        backs = pd.merge(
            back_subset,
            lay_subset,
            on=[
                "market_id",
                "selection_id",
            ],
            how="left",
            suffixes=["_back", "_lay"],
        )

        lays = pd.merge(
            lay_subset,
            back_subset,
            on=[
                "market_id",
                "selection_id",
            ],
            how="left",
            suffixes=["_back", "_lay"],
        )

        data = pd.concat([backs, lays]).drop_duplicates(
            subset=["market_id", "selection_id"]
        )
        data["back_cashout"] = abs(
            data["size_matched_lay"]
            - (
                (data["average_price_matched_back"] * data["size_matched_back"])
                / data["average_price_matched_lay"]
            )
        ).round(2)
        data["lay_cashout"] = abs(
            data["size_matched_back"]
            - (
                (data["average_price_matched_lay"] * data["size_matched_lay"])
                / data["average_price_matched_back"]
            ).round(2)
        )
        return list(
            data[
                ~(
                    (data["lay_cashout"].round(2) == data["back_cashout"].round(2))
                    & (data["lay_cashout"] < 1)
                    & (data["back_cashout"] < 1)
                )
            ]["market_id"].unique()
        )

    def fetch_cash_out_data(self, market_ids: list[str]) -> pd.DataFrame:
        matched_orders = self.get_matched_orders(market_ids)
        market_ids = BetFairClient._get_market_ids_for_remaining_cash_out_bets(
            matched_orders
        )
        if not market_ids:
            return pd.DataFrame()
        current_market_data = self.create_single_market_data(market_ids)
        data = pd.merge(
            matched_orders,
            current_market_data,
            on=["market_id", "selection_id"],
            how="left",
        )
        data = data[data["race_time"] > pd.Timestamp("now", tz="Europe/London")]
        return data[
            [
                "market_id",
                "selection_id",
                "selection_type",
                "average_price_matched",
                "size_matched",
                "market",
                "status",
                "back_price_1",
                "back_price_1_depth",
                "back_price_2",
                "back_price_2_depth",
                "lay_price_1",
                "lay_price_1_depth",
                "lay_price_2",
                "lay_price_2_depth",
            ]
        ]

    def cash_out_bets(self, market_ids: list[str]):
        cashed_out = False
        while not cashed_out:
            cash_out_data = self.fetch_cash_out_data(market_ids)
            if cash_out_data.empty:
                cashed_out = True
            else:
                cash_out_orders = self.betfair_cash_out.cash_out(cash_out_data)
                self.place_orders(cash_out_orders)
                sleep(10)

        return self.get_matched_orders(market_ids)

    def cash_out_bets_for_selection(
        self, market_ids: list[str], selection_ids: list[str]
    ):
        """
        Cash out bets for specific selection IDs within the given markets.

        Args:
            market_ids: List of market IDs to cash out bets from
            selection_ids: List of selection IDs to specifically cash out

        Returns:
            DataFrame of matched orders for the specified selections

        Raises:
            ValueError: If selection_ids is empty or contains invalid values
        """
        if not selection_ids:
            raise ValueError("selection_ids cannot be empty")

        # Convert selection_ids to strings to match data format
        selection_ids_str = [str(sid) for sid in selection_ids]

        I(
            f"Cashing out bets for selections {selection_ids_str} in markets {market_ids}"
        )

        cashed_out = False
        while not cashed_out:
            # Fetch all cash out data for the markets
            cash_out_data = self.fetch_cash_out_data(market_ids)

            if cash_out_data.empty:
                cashed_out = True
                I("No cash out data found")
            else:
                # Filter data to only include the specified selection IDs
                filtered_cash_out_data = cash_out_data[
                    cash_out_data["selection_id"].astype(str).isin(selection_ids_str)
                ]

                if filtered_cash_out_data.empty:
                    cashed_out = True
                    I(f"No bets found for selection IDs {selection_ids_str}")
                else:
                    I(
                        f"Found {len(filtered_cash_out_data)} bets to cash out for selections {selection_ids_str}"
                    )
                    cash_out_orders = self.betfair_cash_out.cash_out(
                        filtered_cash_out_data
                    )

                    if cash_out_orders:
                        I(f"Placing {len(cash_out_orders)} cash out orders")
                        self.place_orders(cash_out_orders)
                        sleep(10)
                    else:
                        cashed_out = True
                        I("No cash out orders generated")

        # Return matched orders filtered by the specified selections
        all_matched_orders = self.get_matched_orders(market_ids)
        if all_matched_orders.empty:
            return all_matched_orders

        return all_matched_orders[
            all_matched_orders["selection_id"].astype(str).isin(selection_ids_str)
        ]

    @staticmethod
    def expand_price_size(data: pd.DataFrame) -> pd.DataFrame:
        price_size_col = "price_size"

        return data.assign(
            price=lambda x: x[price_size_col].apply(lambda x: x["price"]),
            size=lambda x: x[price_size_col].apply(lambda x: x["size"]),
        ).drop(columns=[price_size_col])

    def get_balance(self):
        self.check_session()
        return self.trading_client.account.get_account_funds(
            wallet="UK", lightweight=True
        )["availableToBetBalance"]

    def get_files(self, params: BetfairHistoricalDataParams) -> list[str]:
        self.check_session()
        I(
            f"Fetching historical market data"
            f"From: {params.from_day}-{params.from_month}-{params.from_year}"
            f"To: {params.to_day}-{params.to_month}-{params.to_year}"
        )
        I(params)
        return self.trading_client.historic.get_file_list(
            "Horse Racing",
            "Basic Plan",
            from_day=params.from_day,
            from_month=params.from_month,
            from_year=params.from_year,
            to_day=params.to_day,
            to_month=params.to_month,
            to_year=params.to_year,
            market_types_collection=params.market_types_collection,
            countries_collection=params.countries_collection,
            file_type_collection=params.file_type_collection,
        )

    def fetch_historical_data(
        self,
        file: str,
    ) -> str:
        self.check_session()
        return self.trading_client.historic.download_file(file)

    def _get_order_details(self, bet_id: str) -> dict | None:
        """
        Get order details by bet ID to check matched amounts.

        Args:
            bet_id: The bet ID to look up

        Returns:
            dict with order details or None if not found
        """
        try:
            current_orders = self.trading_client.betting.list_current_orders()

            for order in current_orders.current_orders:
                if order.bet_id == bet_id:
                    return {
                        "matched_size": getattr(order, "size_matched", 0.0),
                        "matched_odds": getattr(order, "average_price_matched", 0.0),
                        "unmatched_size": getattr(order, "size_remaining", 0.0),
                        "status": getattr(order, "status", "UNKNOWN"),
                    }
            return None
        except Exception as e:
            I(f"Error fetching order details for bet {bet_id}: {e}")
            return None


In [108]:
import betfairlightweight
import pandas as pd
import requests
from datetime import datetime, timedelta

trading = betfairlightweight.APIClient(
    username=config.bf_username,
    password=config.bf_password,
    app_key=config.bf_app_key,
    certs=config.bf_certs_path,
)

In [109]:
trading.login(session=requests)


<LoginResource>

In [110]:
import hashlib
import pandas as pd
from api_helpers.helpers.time_utils import get_uk_time_now, make_uk_time_aware
def make_unique_id(race_time, event_id, tz: str | None = None) -> str:
    ts = pd.to_datetime(race_time, errors="coerce")
    if tz is not None and ts is not pd.NaT:
        try:
            ts = ts.tz_localize(tz) if ts.tzinfo is None else ts.tz_convert(tz)
        except Exception:
            pass
    ts_str = "" if ts is pd.NaT else ts.strftime("%Y%m%d%H%M%S")
    key = f"{ts_str}|{'' if event_id is None else str(event_id)}"
    return hashlib.sha256(key.encode("utf-8")).hexdigest()


In [111]:
from betfairlightweight import filters

horse_racing_event_type_filter = filters.market_filter(
    event_type_ids=["7"],
    market_countries=["GB"],
    # market_type_codes=["WIN", "PLACE"],
    market_start_time={
        "from": ((datetime.now()) - timedelta(hours=1)).strftime("%Y-%m-%dT%TZ"),
        "to": (datetime.now())
        .replace(hour=23, minute=59, second=0, microsecond=0)
        .strftime("%Y-%m-%dT%TZ"),
    },
)


In [112]:
events = trading.betting.list_events(
    filter=horse_racing_event_type_filter
)

In [119]:
# --- Fetch the Market Catalogue for ALL Found Events ---
markets = []
if events:
    # Loop through each event we found in the initial search
    for event in events:
        event_id = event.event.id
        event_name = event.event.name
        print(f"\nFetching all markets for event: '{event_name}' (ID: {event_id})")

        # Create a filter for the markets of the current event in the loop
        market_catalogue_filter = filters.market_filter(
            event_ids=[event_id],
        )

        # Request the market catalogue for this specific event
        market_catalogues = trading.betting.list_market_catalogue(
            filter=market_catalogue_filter,
            max_results=100,  # Max number of markets to return per event
            market_projection=['MARKET_DESCRIPTION', 'RUNNER_METADATA', 'EVENT', 'MARKET_START_TIME'],
        )

        for market in market_catalogues:
            for runner in market.runners:

                markets.append(
                    {
                        "event_name": event_name,
                        "event_id": event_id,
                        'race_id': make_unique_id(market.market_start_time, event_id, tz="Europe/London"),
                        'course': market.event.venue,
                        "market_name": market.market_name,
                        "market_id": market.market_id,
                        'race_time': make_uk_time_aware(market.market_start_time),
                        "market_type": market.description.market_type,
                        'race_type': market.description.race_type,
                        "selection_id": runner.selection_id,
                        "horse_name": runner.runner_name,
                        'number_of_runners': len(market.runners),
                    }
            )   


Fetching all markets for event: 'Southwell 21st Sep' (ID: 34751698)

Fetching all markets for event: 'Hamilton 21st Sep' (ID: 34751687)

Fetching all markets for event: 'Plumpton 21st Sep' (ID: 34751691)

Fetching all markets for event: 'Hamilton 21st Sep' (ID: 34751687)

Fetching all markets for event: 'Plumpton 21st Sep' (ID: 34751691)


In [ ]:
def mark_handicaps(data: pd.DataFrame) -> pd.DataFrame:
    race_names = data[data["market_type"] == "WIN"][
        ["market_name", "race_id"]
    ].drop_duplicates()
    handicaps = race_names[race_names["market_name"].str.lower().str.contains("hcap")][
        "race_id"
    ].to_list()

    return data.assign(handicap=np.where(data["race_id"].isin(handicaps), True, False))


def create_market_win_place(data: pd.DataFrame) -> pd.DataFrame:
    conditions = [
        (data["market_type"] == "WIN"),
        (data["market_name"] == "2 TBP"),
        (data["market_name"] == "3 TBP"),
        (data["market_name"] == "4 TBP"),
        (data["market_name"] == "5 TBP"),
        (data["market_name"] == "6 TBP"),
        (
            (data["number_of_runners"].between(3, 4))
            & (data["market_type"] == "PLACE")
            & (data["market_name"] == "To Be Placed")
        ),
        (
            (data["number_of_runners"].between(5, 7))
            & (data["market_type"] == "PLACE")
            & (data["market_name"] == "To Be Placed")
        ),
        (
            (data["number_of_runners"].between(8, 15))
            & (data["market_type"] == "PLACE")
            & (data["market_name"] == "To Be Placed")
        ),
        (
            (data["number_of_runners"] > 15)
            & (data["market_type"] == "PLACE")
            & (data["market_name"] == "To Be Placed")
            & (data["handicap"] == False)
        ),
        (
            (data["number_of_runners"] > 15)
            & (data["market_type"] == "PLACE")
            & (data["market_name"] == "To Be Placed")
            & (data["handicap"] == True)
        ),
    ]

    choices = [1, 2, 3, 4, 5, 6, 1, 2, 3, 3, 4]

    return data.assign(market_win_place=np.select(conditions, choices))

In [ ]:
df = pd.DataFrame(markets)

df = (
    df[df["market_type"].isin(["WIN", "PLACE", "OTHER_PLACE"])]
    .pipe(mark_handicaps)
    .pipe(create_market_win_place)
)
df

# df.sort_values(by=['race_time','market_name']).to_csv('~/Desktop/test2.csv')

,event_name,event_id,race_id,course,market_name,market_id,race_time,market_type,race_type,selection_id,horse_name,number_of_runners,handicap
8,Southwell 21st Sep,34751698,0c971162b26f1a7810162692c5b3a0e9b99632c426d892...,Southwell,1m Mdn Stks,1.247969667,2025-09-21 14:37:00+01:00,WIN,Flat,10097625,Devils Peak,8,False
9,Southwell 21st Sep,34751698,0c971162b26f1a7810162692c5b3a0e9b99632c426d892...,Southwell,1m Mdn Stks,1.247969667,2025-09-21 14:37:00+01:00,WIN,Flat,87999510,Exotic Baby,8,False
10,Southwell 21st Sep,34751698,0c971162b26f1a7810162692c5b3a0e9b99632c426d892...,Southwell,1m Mdn Stks,1.247969667,2025-09-21 14:37:00+01:00,WIN,Flat,88794708,Only In Manila,8,False
11,Southwell 21st Sep,34751698,0c971162b26f1a7810162692c5b3a0e9b99632c426d892...,Southwell,1m Mdn Stks,1.247969667,2025-09-21 14:37:00+01:00,WIN,Flat,88794707,Ruthless Ambition,8,False
12,Southwell 21st Sep,34751698,0c971162b26f1a7810162692c5b3a0e9b99632c426d892...,Southwell,1m Mdn Stks,1.247969667,2025-09-21 14:37:00+01:00,WIN,Flat,87177914,Thatcham,8,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,Plumpton 21st Sep,34751691,23361359e4df6cb7518e59c1b170c326d1414c36cf1244...,Plumpton,2m2f NHF,1.247969604,2025-09-21 16:50:00+01:00,WIN,NHF,78060234,Masked Mistress,5,False
905,Plumpton 21st Sep,34751691,23361359e4df6cb7518e59c1b170c326d1414c36cf1244...,Plumpton,2m2f NHF,1.247969604,2025-09-21 16:50:00+01:00,WIN,NHF,85452844,Nelrose,5,False
906,Plumpton 21st Sep,34751691,23361359e4df6cb7518e59c1b170c326d1414c36cf1244...,Plumpton,2m2f NHF,1.247969604,2025-09-21 16:50:00+01:00,WIN,NHF,88794705,Arms Park,5,False
907,Plumpton 21st Sep,34751691,23361359e4df6cb7518e59c1b170c326d1414c36cf1244...,Plumpton,2m2f NHF,1.247969604,2025-09-21 16:50:00+01:00,WIN,NHF,84484374,Mushtahrir,5,False


In [ ]:
df[df["race_time"] == "2025-09-21 16:50:00+01:00"]

"""


    if number_of_runners <= 4:
        return 1  # Win only
    elif number_of_runners <= 7:
        return 2  # Pays first 2
    elif number_of_runners <= 15:
        return 3  # Pays first 3
    # Logic for 16 or more runners
    elif is_handicap:
        return 4  # Pays first 4 in a handicap
    else:
        return 3  # Pays first 3 in a non-handicap

        
        """

conditions = [
    (df["market_type"] == "WIN"),
    (df["market_name"] == "2 TBP"),
    (df["market_name"] == "3 TBP"),
    (df["market_name"] == "4 TBP"),
    (df["market_name"] == "5 TBP"),
    (df["market_name"] == "6 TBP"),
    (
        (df["number_of_runners"].between(3, 4))
        & (df["market_type"] == "PLACE")
        & (df["market_name"] == "To Be Placed")
    ),
    (
        (df["number_of_runners"].between(5, 7))
        & (df["market_type"] == "PLACE")
        & (df["market_name"] == "To Be Placed")
    ),
    (
        (df["number_of_runners"].between(8, 15))
        & (df["market_type"] == "PLACE")
        & (df["market_name"] == "To Be Placed")
    ),
    (
        (df["number_of_runners"] > 15)
        & (df["market_type"] == "PLACE")
        & (df["market_name"] == "To Be Placed")
        & (df['handicap'] == False) 
    ),
    (
        (df["number_of_runners"] > 15)
        & (df["market_type"] == "PLACE")
        & (df["market_name"] == "To Be Placed")
        & (df['handicap'] == True)
    ),
]

choices = [1, 2, 3, 4, 5, 6, 1, 2, 3, 3, 4]

df['market_win_place'] = np.select(conditions, choices)

df.to_csv('~/Desktop/test3.csv', index=False)

In [105]:
p = df.drop_duplicates(subset=["race_time"]).sort_values(by=['race_time','market_name'])[
    [
        'race_id',
        'race_time',
        "market_name",
        "market_type",
        'market_id',
        "number_of_runners",
    ]
]

p

,race_id,race_time,market_name,market_type,market_id,number_of_runners
10,bbad49bbffd9106a53b26742cc1958301c82b676d7e153...,2025-09-20 13:00:00+01:00,1m Nov Stks,WIN,1.247938605,8
1645,76c42e40ae100950a6e910c11f339a538a4333afa646b8...,2025-09-20 13:15:00+01:00,1m Hcap,WIN,1.247939515,14
866,0fbd6237fcab334a7eb5f66c7af4682cb6a7247c552883...,2025-09-20 13:30:00+01:00,5f Grp 3,WIN,1.247938537,11
53,d198efcea63bcce385801bc808a70d306b752e75520db8...,2025-09-20 13:35:00+01:00,1m Hcap,WIN,1.247938612,9
1705,a6497d75dc1b99fd8aceebd7318ed3b2c9587911a0b9e5...,2025-09-20 13:50:00+01:00,1m2f Listed,WIN,1.247939521,6
2632,0f65e1be005404ee25513dd8fd1c20c45c1bffc86c195a...,2025-09-20 13:55:00+01:00,7f Nursery,WIN,1.247938489,9
928,e20a8f210633e57855320c79848bc687f6946442ce845b...,2025-09-20 14:05:00+01:00,1m5f Hcap,WIN,1.247938543,14
89,a3ee1a158be7af287644c0d2360c5149b7713d5bebdf28...,2025-09-20 14:10:00+01:00,4 TBP,OTHER_PLACE,1.247938713,13
1729,e0fa746d0468889f07fa16729772d6f4deaf64c1bc3908...,2025-09-20 14:25:00+01:00,5 TBP,OTHER_PLACE,1.247939532,25
2670,e77a631d029e51db1144b88a6f79ceb95c09eb0b6ee37a...,2025-09-20 14:30:00+01:00,6f Nov Stks,WIN,1.247938495,6


In [ ]:
def get_places(number_of_runners: int, is_handicap: bool = False) -> int:
    """
    Calculates the number of paid places in a UK horse race.

    Args:
        number_of_runners: The total number of runners in the race.
        is_handicap: True if the race is a handicap, False otherwise.

    Returns:
        The number of places paid.
    """
    if number_of_runners <= 4:
        return 1  # Win only
    elif number_of_runners <= 7:
        return 2  # Pays first 2
    elif number_of_runners <= 15:
        return 3  # Pays first 3
    # Logic for 16 or more runners
    elif is_handicap:
        return 4  # Pays first 4 in a handicap
    else:
        return 3  # Pays first 3 in a non-handicap

mapping = []

for race_id in p['race_id']:
    race = p[p['race_id'] == race_id]
    market_names = race['market_name'].tolist()
    market_types = sorted(set(race['market_types'].to_list()))
    if (len(market_names) == 4) and (market_types == {'OTHER_PLACE','PLACE', 'WIN'}):
        mapping.append(
            {

            }
        )

,race_id,market_name,market_type,number_of_runners
10,bbad49bbffd9106a53b26742cc1958301c82b676d7e153...,1m Nov Stks,WIN,8
26,bbad49bbffd9106a53b26742cc1958301c82b676d7e153...,2 TBP,OTHER_PLACE,8
34,bbad49bbffd9106a53b26742cc1958301c82b676d7e153...,4 TBP,OTHER_PLACE,8
18,bbad49bbffd9106a53b26742cc1958301c82b676d7e153...,To Be Placed,PLACE,8
1645,76c42e40ae100950a6e910c11f339a538a4333afa646b8...,1m Hcap,WIN,14
1673,76c42e40ae100950a6e910c11f339a538a4333afa646b8...,2 TBP,OTHER_PLACE,14
1687,76c42e40ae100950a6e910c11f339a538a4333afa646b8...,4 TBP,OTHER_PLACE,14
1659,76c42e40ae100950a6e910c11f339a538a4333afa646b8...,To Be Placed,PLACE,14
888,0fbd6237fcab334a7eb5f66c7af4682cb6a7247c552883...,2 TBP,OTHER_PLACE,11
899,0fbd6237fcab334a7eb5f66c7af4682cb6a7247c552883...,4 TBP,OTHER_PLACE,11


In [ ]:

# Single values
# unique_id = make_unique_id(race_time_value, event_id_value)

# If you later want to fill a column from a DataFrame:
# df["unique_id"] = df.apply(lambda r: make_unique_id(r["race_time"], r["event_id"]), axis=1)

In [59]:
df['market_name'].value_counts()

market_name
To Be Placed     431
4 TBP            355
2 TBP            325
6f Hcap           83
3 TBP             76
7f Hcap           74
6 TBP             50
5 TBP             50
1m Hcap           38
1m1f Hcap         26
7f Mdn Stks       24
7f Nov Stks       20
1m2f Hcap         20
5f Hcap           19
6f Grp 3          15
1m5f Hcap         14
1m6f Hcap         14
5f Grp 3          11
1m2f Nov Stks      9
7f Nursery         9
7f Stks            9
2m2f Hcap          9
6f Grp 2           8
5f Nursery         8
1m Nov Stks        8
1m4f Hcap          7
1m2f Listed        6
6f Nov Stks        6
1m1f Nursery       3
Name: count, dtype: int64

In [42]:
def calculate_win_place_market_type(market_name: str, number_of_runners: int) -> int:
    """
    Calculate the number of places for a 'To Be Placed' market based on the market name and number of runners.

    Args:
        market_name (str): The name of the market, e.g., "To Be Placed 1-3 (4+ Runners) TBP".
        number_of_runners (int): The total number

    Returns:
        int: The number of places for the market.

    """

    

market_name
To Be Placed     38
4 TBP            32
2 TBP            30
3 TBP             6
7f Hcap           6
6f Hcap           5
1m Hcap           3
7f Mdn Stks       2
1m2f Hcap         2
5 TBP             2
6 TBP             2
1m1f Hcap         2
7f Nov Stks       2
5f Hcap           2
7f Nursery        1
6f Grp 3          1
6f Nov Stks       1
7f Stks           1
1m2f Listed       1
1m Nov Stks       1
6f Grp 2          1
1m5f Hcap         1
5f Grp 3          1
1m4f Hcap         1
5f Nursery        1
1m1f Nursery      1
1m2f Nov Stks     1
2m2f Hcap         1
1m6f Hcap         1
Name: count, dtype: int64

In [43]:
df.to_csv('~/Desktop/test.csv')

In [118]:
markets = []
if events:
    # Loop through each event we found in the initial search
    for event in events:
        event_id = event.event.id
        event_name = event.event.name
        print(f"\nFetching all markets for event: '{event_name}' (ID: {event_id})")

        # Create a filter for the markets of the current event in the loop
        market_catalogue_filter = filters.market_filter(
            event_ids=[event_id],
        )

        # Request the market catalogue for this specific event
        market_catalogues = trading.betting.list_market_catalogue(
            filter=market_catalogue_filter,
            max_results=100,  # Max number of markets to return per event
            market_projection=['MARKET_DESCRIPTION', 'RUNNER_METADATA']
        )

        for market in market_catalogues:
            for runner in market.runners:

                markets.append(
                    {
                        "event_name": event_name,
                        "event_id": event_id,
                        "market_name": market.market_name,
                        "market_id": market.market_id,
                        'race_time': market.market_start_time,
                        "market_type": market.description.market_type,
                        "selection_id": runner.selection_id,
                        "horse_name": runner.runner_name,
                    }
            )   



Fetching all markets for event: 'Southwell 21st Sep' (ID: 34751698)

Fetching all markets for event: 'Hamilton 21st Sep' (ID: 34751687)

Fetching all markets for event: 'Plumpton 21st Sep' (ID: 34751691)

Fetching all markets for event: 'Plumpton 21st Sep' (ID: 34751691)


In [ ]:
# Create a SHA-256 unique_id from race_time and event_id
import hashlib
import pandas as pd

def add_unique_id_from_race_time_and_event_id(
    df: pd.DataFrame,
    race_time_col: str = "race_time",
    event_id_col: str = "event_id",
    tz: str | None = None,
 ) -> pd.DataFrame:
    """
    Return a copy of df with a new 'unique_id' column computed as:
        sha256(f"{race_time:%Y%m%d%H%M%S}|{event_id}")
    
    Args:
        df: Input DataFrame containing race_time and event_id.
        race_time_col: Name of the race_time column.
        event_id_col: Name of the event_id column.
        tz: Optional timezone name (e.g., 'Europe/London') to normalize race_time before hashing.
    """
    d = df.copy()
    rt = pd.to_datetime(d[race_time_col], errors="coerce")
    
    # Optionally normalize timezone for deterministic hashing
    if tz is not None:
        try:
            if rt.dt.tz is None:
                rt = rt.dt.tz_localize(tz)
            else:
                rt = rt.dt.tz_convert(tz)
        except Exception:
            # If tz conversion/localization fails, fall back to original parsed times
            pass
    
    race_time_str = rt.dt.strftime("%Y%m%d%H%M%S").fillna("")
    key_series = race_time_str + "|" + d[event_id_col].astype(str).fillna("")
    d["unique_id"] = key_series.map(lambda s: hashlib.sha256(s.encode("utf-8")).hexdigest())
    return d

# Example usage:
# df = add_unique_id_from_race_time_and_event_id(df)


In [ ]:
{
    "marketId": "1.247969675",
    "marketName": "To Be Placed",
    "marketStartTime": "2025-09-21T14:07:00.000Z",
    "description": {
        "persistenceEnabled": True,
        "bspMarket": True,
        "marketTime": "2025-09-21T14:07:00.000Z",
        "suspendTime": "2025-09-21T14:07:00.000Z",
        "bettingType": "ODDS",
        "turnInPlayEnabled": True,
        "marketType": "PLACE",
        "regulator": "MALTA LOTTERIES AND GAMBLING AUTHORITY",
        "marketBaseRate": 2.0,
        "discountAllowed": False,
        "wallet": "UK wallet",
        "rules": '<br><a href="https://www.timeform.com/horse-racing/" target="_blank"><img src=" http://content-cache.betfair.com/images/en_GB/mr_fr.gif" title=”Form/ Results” border="0"></a>\n<br><br><b>MARKET INFORMATION</b><br><br>For further information please see <a href=http://content.betfair.com/aboutus/content.asp?sWhichKey=Rules%20and%20Regulations#undefined.do style=color:0163ad; text-decoration: underline; target=_blank>Rules & Regs.</a><br><br>Who will finish 1st, 2nd or 3rd in this race? NON RUNNERS DO NOT CHANGE THE PLACE TERMS. Should the number of runners be equal to or less than the number of places available as set out above in these rules all bets will be void. Betfair Non-Runner Rule applies. <b>This market will turn IN PLAY at the off with unmatched bets (with the exception of bets for which the "keep" option has been selected) cancelled once the Betfair SP reconciliation process has been completed. Betting will be suspended at the end of the race.</b> This market will initially be settled on a First Past the Post basis. However we will re-settle all bets should the official result at the time of the "weigh-in" announcement differ from any initial settlement. BETS ARE PLACED ON A NAMED HORSE. Dead Heat rules apply.<br><br>Customers should be aware that:<ol><b><li>transmissions described as "live" by some broadcasters may actually be delayed;</li><li>the extent of any such delay may vary, depending on the set-up through which they are receiving pictures or data; and</li></b><li>information (such as jockey silks, saddlecloth numbers etc) is provided "as is" and is for guidance only. Betfair does not guarantee the accuracy of this information and use of it to place bets is entirely at your own risk.</li></ol><br>',
        "rulesHasDate": True,
        "raceType": "Flat",
        "priceLadderDescription": {"type": "CLASSIC"},
    },
    "totalMatched": 2.1,
    "runners": [
        {
            "selectionId": 50612968,
            "runnerName": "Dragon Icon",
            "handicap": 0.0,
            "sortPriority": 1,
            "metadata": {
                "SIRE_NAME": "Lope De Vega",
                "CLOTH_NUMBER_ALPHA": "2",
                "OFFICIAL_RATING": "81",
                "COLOURS_DESCRIPTION": "Blue, red sleeves",
                "COLOURS_FILENAME": "c20250921sou/00881392.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Hurricane Run",
                "WEIGHT_VALUE": "136",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "15",
                "WEARING": "Cheek pieces",
                "OWNER_NAME": "Mrs Angelica Spence",
                "DAM_YEAR_BORN": "2009",
                "SIRE_BRED": "IRL",
                "JOCKEY_NAME": "Ray Dawson",
                "DAM_BRED": "IRL",
                "ADJUSTED_RATING": None,
                "runnerId": "50612968",
                "CLOTH_NUMBER": "2",
                "SIRE_YEAR_BORN": "2007",
                "TRAINER_NAME": "R Varian",
                "COLOUR_TYPE": "b",
                "AGE": "5",
                "DAMSIRE_BRED": "IRL",
                "JOCKEY_CLAIM": "0",
                "FORM": "350733",
                "FORECASTPRICE_NUMERATOR": "6",
                "BRED": None,
                "DAM_NAME": "Matauri Pearl",
                "DAMSIRE_YEAR_BORN": "2002",
                "STALL_DRAW": "1",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 74754399,
            "runnerName": "Best Rate",
            "handicap": 0.0,
            "sortPriority": 2,
            "metadata": {
                "SIRE_NAME": "Camacho",
                "CLOTH_NUMBER_ALPHA": "10",
                "OFFICIAL_RATING": "82",
                "COLOURS_DESCRIPTION": "Black and yellow diamonds, black sleeves, yellow cap",
                "COLOURS_FILENAME": "c20250921sou/00881778.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Royal Applause",
                "WEIGHT_VALUE": "133",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "16",
                "WEARING": None,
                "OWNER_NAME": "Jonathan Palmer-brown And Jim Horgan",
                "DAM_YEAR_BORN": "2010",
                "SIRE_BRED": "GBR",
                "JOCKEY_NAME": "Tom Marquand",
                "DAM_BRED": "GBR",
                "ADJUSTED_RATING": None,
                "runnerId": "74754399",
                "CLOTH_NUMBER": "10",
                "SIRE_YEAR_BORN": "2002",
                "TRAINER_NAME": "R Hannon",
                "COLOUR_TYPE": "b",
                "AGE": "3",
                "DAMSIRE_BRED": "GBR",
                "JOCKEY_CLAIM": "0",
                "FORM": "204423",
                "FORECASTPRICE_NUMERATOR": "6",
                "BRED": None,
                "DAM_NAME": "Nardin",
                "DAMSIRE_YEAR_BORN": "1993",
                "STALL_DRAW": "7",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 47296230,
            "runnerName": "Rajapour",
            "handicap": 0.0,
            "sortPriority": 3,
            "metadata": {
                "SIRE_NAME": "Almanzor",
                "CLOTH_NUMBER_ALPHA": "8",
                "OFFICIAL_RATING": "79",
                "COLOURS_DESCRIPTION": "White, red inverted triangle, hooped sleeves and cap",
                "COLOURS_FILENAME": "c20250921sou/00062867.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Rock Of Gibraltar",
                "WEIGHT_VALUE": "134",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "14",
                "WEARING": None,
                "OWNER_NAME": "Mr Evan M Sutherland",
                "DAM_YEAR_BORN": "2012",
                "SIRE_BRED": "FRA",
                "JOCKEY_NAME": "Mark Winn",
                "DAM_BRED": "IRL",
                "ADJUSTED_RATING": None,
                "runnerId": "47296230",
                "CLOTH_NUMBER": "8",
                "SIRE_YEAR_BORN": "1994",
                "TRAINER_NAME": "D O'Meara",
                "COLOUR_TYPE": "ch",
                "AGE": "5",
                "DAMSIRE_BRED": "IRL",
                "JOCKEY_CLAIM": "0",
                "FORM": "6769-42",
                "FORECASTPRICE_NUMERATOR": "7",
                "BRED": None,
                "DAM_NAME": "Raydara",
                "DAMSIRE_YEAR_BORN": "1999",
                "STALL_DRAW": "11",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 71597397,
            "runnerName": "Degale",
            "handicap": 0.0,
            "sortPriority": 4,
            "metadata": {
                "SIRE_NAME": "Due Diligence",
                "CLOTH_NUMBER_ALPHA": "4",
                "OFFICIAL_RATING": "80",
                "COLOURS_DESCRIPTION": "White, emerald green cross of lorraine,  chevrons on sleeves, pink cap",
                "COLOURS_FILENAME": "c20250921sou/00848182.jpg",
                "FORECASTPRICE_DENOMINATOR": "2",
                "DAMSIRE_NAME": "Broken Vow",
                "WEIGHT_VALUE": "135",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "42",
                "WEARING": None,
                "OWNER_NAME": "Mrlaurenceo'kane/harrowgatebloodstockl",
                "DAM_YEAR_BORN": "1998",
                "SIRE_BRED": "USA",
                "JOCKEY_NAME": "Callum Rodriguez",
                "DAM_BRED": "USA",
                "ADJUSTED_RATING": None,
                "runnerId": "71597397",
                "CLOTH_NUMBER": "4",
                "SIRE_YEAR_BORN": "2011",
                "TRAINER_NAME": "T D Barron",
                "COLOUR_TYPE": "b",
                "AGE": "4",
                "DAMSIRE_BRED": "USA",
                "JOCKEY_CLAIM": "0",
                "FORM": "831-2",
                "FORECASTPRICE_NUMERATOR": "11",
                "BRED": None,
                "DAM_NAME": "Nuptials",
                "DAMSIRE_YEAR_BORN": "1997",
                "STALL_DRAW": "6",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 57120811,
            "runnerName": "My Margie",
            "handicap": 0.0,
            "sortPriority": 5,
            "metadata": {
                "SIRE_NAME": "Dandy Man",
                "CLOTH_NUMBER_ALPHA": "5",
                "OFFICIAL_RATING": "80",
                "COLOURS_DESCRIPTION": "Red, large yellow spots, armlets and spots on cap",
                "COLOURS_FILENAME": "c20250921sou/00885004.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Dynaformer",
                "WEIGHT_VALUE": "135",
                "SEX_TYPE": "F",
                "DAYS_SINCE_LAST_RUN": "25",
                "WEARING": None,
                "OWNER_NAME": "Mrs M Gander",
                "DAM_YEAR_BORN": "2007",
                "SIRE_BRED": "IRL",
                "JOCKEY_NAME": "Finley Marsh",
                "DAM_BRED": "USA",
                "ADJUSTED_RATING": None,
                "runnerId": "57120811",
                "CLOTH_NUMBER": "5",
                "SIRE_YEAR_BORN": "2003",
                "TRAINER_NAME": "R Hughes",
                "COLOUR_TYPE": "b",
                "AGE": "4",
                "DAMSIRE_BRED": "USA",
                "JOCKEY_CLAIM": "0",
                "FORM": "165304",
                "FORECASTPRICE_NUMERATOR": "16",
                "BRED": None,
                "DAM_NAME": "Agapantha",
                "DAMSIRE_YEAR_BORN": "1985",
                "STALL_DRAW": "3",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 394588,
            "runnerName": "Great Blasket",
            "handicap": 0.0,
            "sortPriority": 6,
            "metadata": {
                "SIRE_NAME": "Gregorian",
                "CLOTH_NUMBER_ALPHA": "9",
                "OFFICIAL_RATING": "79",
                "COLOURS_DESCRIPTION": "Orange, orange and black check sleeves",
                "COLOURS_FILENAME": "c20250921sou/00881512.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Oasis Dream",
                "WEIGHT_VALUE": "134",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "22",
                "WEARING": "Blinkers",
                "OWNER_NAME": "Urloxhey Racing",
                "DAM_YEAR_BORN": "0",
                "SIRE_BRED": "IRL",
                "JOCKEY_NAME": "Hollie Doyle",
                "DAM_BRED": "GBR",
                "ADJUSTED_RATING": None,
                "runnerId": "394588",
                "CLOTH_NUMBER": "9",
                "SIRE_YEAR_BORN": "1997",
                "TRAINER_NAME": "Dr R Newland & J Insole",
                "COLOUR_TYPE": "b",
                "AGE": "5",
                "DAMSIRE_BRED": "GBR",
                "JOCKEY_CLAIM": "0",
                "FORM": "018163",
                "FORECASTPRICE_NUMERATOR": "7",
                "BRED": None,
                "DAM_NAME": "Dream Belle",
                "DAMSIRE_YEAR_BORN": "2000",
                "STALL_DRAW": "10",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 71352621,
            "runnerName": "Man Of Desert",
            "handicap": 0.0,
            "sortPriority": 7,
            "metadata": {
                "SIRE_NAME": "Study Of Man",
                "CLOTH_NUMBER_ALPHA": "13",
                "OFFICIAL_RATING": "75",
                "COLOURS_DESCRIPTION": "White, red hollow box",
                "COLOURS_FILENAME": "c20250921sou/00067209.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Green Desert",
                "WEIGHT_VALUE": "130",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "59",
                "WEARING": None,
                "OWNER_NAME": "Strawberry Fields Stud",
                "DAM_YEAR_BORN": "2009",
                "SIRE_BRED": "IRL",
                "JOCKEY_NAME": "Dylan Hogan",
                "DAM_BRED": "GBR",
                "ADJUSTED_RATING": None,
                "runnerId": "71352621",
                "CLOTH_NUMBER": "13",
                "SIRE_YEAR_BORN": "2015",
                "TRAINER_NAME": "C A Dwyer",
                "COLOUR_TYPE": "b",
                "AGE": "4",
                "DAMSIRE_BRED": "USA",
                "JOCKEY_CLAIM": "0",
                "FORM": "6-63048",
                "FORECASTPRICE_NUMERATOR": "14",
                "BRED": None,
                "DAM_NAME": "Desert Berry",
                "DAMSIRE_YEAR_BORN": "1983",
                "STALL_DRAW": "4",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 23983741,
            "runnerName": "Dutch Decoy",
            "handicap": 0.0,
            "sortPriority": 8,
            "metadata": {
                "SIRE_NAME": "Dutch Art",
                "CLOTH_NUMBER_ALPHA": "1",
                "OFFICIAL_RATING": "81",
                "COLOURS_DESCRIPTION": "Mauve, black chevrons on sleeves, black cap",
                "COLOURS_FILENAME": "c20250921sou/00863054.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Foxhound",
                "WEIGHT_VALUE": "136",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "31",
                "WEARING": "Cheek pieces",
                "OWNER_NAME": "Owners Group 052",
                "DAM_YEAR_BORN": "2003",
                "SIRE_BRED": "GBR",
                "JOCKEY_NAME": "J Mitchell",
                "DAM_BRED": "GBR",
                "ADJUSTED_RATING": None,
                "runnerId": "23983741",
                "CLOTH_NUMBER": "1",
                "SIRE_YEAR_BORN": "2004",
                "TRAINER_NAME": "C Johnston",
                "COLOUR_TYPE": "ch",
                "AGE": "8",
                "DAMSIRE_BRED": "USA",
                "JOCKEY_CLAIM": "0",
                "FORM": "110050",
                "FORECASTPRICE_NUMERATOR": "12",
                "BRED": None,
                "DAM_NAME": "The Terrier",
                "DAMSIRE_YEAR_BORN": "1991",
                "STALL_DRAW": "2",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 48298473,
            "runnerName": "Leadenhall",
            "handicap": 0.0,
            "sortPriority": 9,
            "metadata": {
                "SIRE_NAME": "Kingman",
                "CLOTH_NUMBER_ALPHA": "12",
                "OFFICIAL_RATING": "76",
                "COLOURS_DESCRIPTION": "Pink, dark blue star, dark blue cap",
                "COLOURS_FILENAME": "c20250921sou/00867647.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Danehill",
                "WEIGHT_VALUE": "131",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "1",
                "WEARING": None,
                "OWNER_NAME": "The Wolf Pack 2 And Partner",
                "DAM_YEAR_BORN": "2004",
                "SIRE_BRED": "GBR",
                "JOCKEY_NAME": "Sean Kirrane",
                "DAM_BRED": "GBR",
                "ADJUSTED_RATING": None,
                "runnerId": "48298473",
                "CLOTH_NUMBER": "12",
                "SIRE_YEAR_BORN": "2011",
                "TRAINER_NAME": "T D Easterby",
                "COLOUR_TYPE": "b",
                "AGE": "5",
                "DAMSIRE_BRED": "USA",
                "JOCKEY_CLAIM": "0",
                "FORM": "041446",
                "FORECASTPRICE_NUMERATOR": "8",
                "BRED": None,
                "DAM_NAME": "Promising Lead",
                "DAMSIRE_YEAR_BORN": "1986",
                "STALL_DRAW": "5",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 47648763,
            "runnerName": "Chalk Mountain",
            "handicap": 0.0,
            "sortPriority": 10,
            "metadata": {
                "SIRE_NAME": "Outstrip",
                "CLOTH_NUMBER_ALPHA": "14",
                "OFFICIAL_RATING": "75",
                "COLOURS_DESCRIPTION": "Maroon, beige chevron, sleeves and star on cap",
                "COLOURS_FILENAME": "c20250921sou/00874683.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Oasis Dream",
                "WEIGHT_VALUE": "130",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "16",
                "WEARING": "Tongue strap,Visor",
                "OWNER_NAME": "The Chalk Mountain Partnership",
                "DAM_YEAR_BORN": "2010",
                "SIRE_BRED": "GBR",
                "JOCKEY_NAME": "Rob Hornby",
                "DAM_BRED": "GBR",
                "ADJUSTED_RATING": None,
                "runnerId": "47648763",
                "CLOTH_NUMBER": "14",
                "SIRE_YEAR_BORN": "2011",
                "TRAINER_NAME": "W S Kittow",
                "COLOUR_TYPE": "gr",
                "AGE": "5",
                "DAMSIRE_BRED": "GBR",
                "JOCKEY_CLAIM": "0",
                "FORM": "234774",
                "FORECASTPRICE_NUMERATOR": "12",
                "BRED": None,
                "DAM_NAME": "Perfect Muse",
                "DAMSIRE_YEAR_BORN": "2000",
                "STALL_DRAW": "13",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 42717141,
            "runnerName": "Borgi",
            "handicap": 0.0,
            "sortPriority": 11,
            "metadata": {
                "SIRE_NAME": "Anjaal",
                "CLOTH_NUMBER_ALPHA": "3",
                "OFFICIAL_RATING": "81",
                "COLOURS_DESCRIPTION": "Black, yellow seams and armlets",
                "COLOURS_FILENAME": "c20250921sou/00005751.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Olden Times",
                "WEIGHT_VALUE": "136",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "38",
                "WEARING": None,
                "OWNER_NAME": "Mr S P C Woods",
                "DAM_YEAR_BORN": "1995",
                "SIRE_BRED": "GBR",
                "JOCKEY_NAME": "Jack Callan",
                "DAM_BRED": "IRL",
                "ADJUSTED_RATING": None,
                "runnerId": "42717141",
                "CLOTH_NUMBER": "3",
                "SIRE_YEAR_BORN": "2011",
                "TRAINER_NAME": "S Woods",
                "COLOUR_TYPE": "b",
                "AGE": "6",
                "DAMSIRE_BRED": "GBR",
                "JOCKEY_CLAIM": "5",
                "FORM": "410-596",
                "FORECASTPRICE_NUMERATOR": "20",
                "BRED": None,
                "DAM_NAME": "One Time",
                "DAMSIRE_YEAR_BORN": "1998",
                "STALL_DRAW": "8",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 48500511,
            "runnerName": "Caragio",
            "handicap": 0.0,
            "sortPriority": 12,
            "metadata": {
                "SIRE_NAME": "Caravaggio",
                "CLOTH_NUMBER_ALPHA": "11",
                "OFFICIAL_RATING": "77",
                "COLOURS_DESCRIPTION": "White, red stars, diabolo on sleeves and star on cap",
                "COLOURS_FILENAME": "c20250921sou/00881965.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "More Than Ready",
                "WEIGHT_VALUE": "132",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "485",
                "WEARING": None,
                "OWNER_NAME": "The Poor Sexy Farmers Club",
                "DAM_YEAR_BORN": "2007",
                "SIRE_BRED": "USA",
                "JOCKEY_NAME": "Tyler Heard",
                "DAM_BRED": "USA",
                "ADJUSTED_RATING": None,
                "runnerId": "48500511",
                "CLOTH_NUMBER": "11",
                "SIRE_YEAR_BORN": "2014",
                "TRAINER_NAME": "Martin Dunne",
                "COLOUR_TYPE": "gr",
                "AGE": "5",
                "DAMSIRE_BRED": "USA",
                "JOCKEY_CLAIM": "0",
                "FORM": "187/400-",
                "FORECASTPRICE_NUMERATOR": "25",
                "BRED": None,
                "DAM_NAME": "Freddies Girl",
                "DAMSIRE_YEAR_BORN": "1997",
                "STALL_DRAW": "9",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 51494,
            "runnerName": "Kalamunda",
            "handicap": 0.0,
            "sortPriority": 13,
            "metadata": {
                "SIRE_NAME": "Zoustar",
                "CLOTH_NUMBER_ALPHA": "7",
                "OFFICIAL_RATING": "79",
                "COLOURS_DESCRIPTION": "White, maroon stars, maroon sleeves, white stars, white cap, maroon star",
                "COLOURS_FILENAME": "c20250921sou/00860614.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "War Chant",
                "WEIGHT_VALUE": "134",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "29",
                "WEARING": "Cheek pieces",
                "OWNER_NAME": "Trevor And Ruth Milner",
                "DAM_YEAR_BORN": "2002",
                "SIRE_BRED": "AUS",
                "JOCKEY_NAME": "Daniel Muscutt",
                "DAM_BRED": "USA",
                "ADJUSTED_RATING": None,
                "runnerId": "51494",
                "CLOTH_NUMBER": "7",
                "SIRE_YEAR_BORN": "2010",
                "TRAINER_NAME": "J Parr",
                "COLOUR_TYPE": "b",
                "AGE": "5",
                "DAMSIRE_BRED": "USA",
                "JOCKEY_CLAIM": "0",
                "FORM": "454044",
                "FORECASTPRICE_NUMERATOR": "20",
                "BRED": None,
                "DAM_NAME": "Karens Caper",
                "DAMSIRE_YEAR_BORN": "1997",
                "STALL_DRAW": "14",
                "WEIGHT_UNITS": "pounds",
            },
        },
        {
            "selectionId": 56809975,
            "runnerName": "Zryan",
            "handicap": 0.0,
            "sortPriority": 14,
            "metadata": {
                "SIRE_NAME": "Night Of Thunder",
                "CLOTH_NUMBER_ALPHA": "6",
                "OFFICIAL_RATING": "80",
                "COLOURS_DESCRIPTION": "Emerald green, white hoops, white sleeves, emerald green diamonds, white cap, emerald green diamond",
                "COLOURS_FILENAME": "c20250921sou/00836517.jpg",
                "FORECASTPRICE_DENOMINATOR": "1",
                "DAMSIRE_NAME": "Green Desert",
                "WEIGHT_VALUE": "135",
                "SEX_TYPE": "G",
                "DAYS_SINCE_LAST_RUN": "14",
                "WEARING": None,
                "OWNER_NAME": "Gallop Racing",
                "DAM_YEAR_BORN": "2006",
                "SIRE_BRED": "IRL",
                "JOCKEY_NAME": "James Doyle",
                "DAM_BRED": "GBR",
                "ADJUSTED_RATING": None,
                "runnerId": "56809975",
                "CLOTH_NUMBER": "6",
                "SIRE_YEAR_BORN": "2011",
                "TRAINER_NAME": "D O'Meara",
                "COLOUR_TYPE": "b",
                "AGE": "4",
                "DAMSIRE_BRED": "USA",
                "JOCKEY_CLAIM": "0",
                "FORM": "149040",
                "FORECASTPRICE_NUMERATOR": "14",
                "BRED": None,
                "DAM_NAME": "Dahama",
                "DAMSIRE_YEAR_BORN": "1983",
                "STALL_DRAW": "12",
                "WEIGHT_UNITS": "pounds",
            },
        },
    ],
    "event": {
        "id": "34751698",
        "name": "Southwell 21st Sep",
        "countryCode": "GB",
        "timezone": "Europe/London",
        "venue": "Southwell",
        "openDate": "2025-09-21T13:37:00.000Z",
    },
}